# Classification
### Step 1: resize/crop images
Requires having run: `beescrape.py`
Depends on yolov5 and `shutil`

In [1]:
# init
import os
from os import listdir
from shutil import copyfile
import shutil
from PIL import Image
import skimage
import random
import numpy as np

# Directories
DATA_DIR = '/home/wilber/Documents/RESEARCH/research/beespotter'
OUT_DIR = '/m2docs/res/data'
CROPPED_PATH = '/m2docs/res/cropped_imgs'
cropped_files = listdir(CROPPED_PATH)

# Args:
VAL_SIZE = 256
TEST_SIZE = 128
preprocess = True
resize = True
length = 256
size = (length, length) # (512, 512)

# Uncropped args: (use the same images except without passing through yolo bee finder for comparison)
OUT_DIR_RAW = '/m2docs/res/data_raw'
UNCROPPED_PATH = '/m2docs/res/uncropped_imgs'
COPY_UNCROPPED = True

# Species with >= 1000 images:
classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']


In [2]:
%rm -R /m2docs/res/data/*

# from https://gist.github.com/Prasad9/28f6a2df8e8d463c6ddd040f4f6a028a
noise_modes = [None,'salt','pepper','s&p']
NOISE_AMOUNT = .01 # default amount
def add_noise(img, mode, noise_amount = .03):
    if mode is not None:
        gimg = skimage.util.random_noise(img, mode = mode, amount = random.uniform(0,noise_amount))
        return gimg
    else:
        print("oops, you shouldn't see this")

total_skipped = 0
for label in classes:  # for each type of bee
    skipnum = 0
    classID = classes.index(label)
    
    input_path = os.path.join(DATA_DIR, label)
    im_list = os.listdir(input_path)
    random.shuffle(im_list)
    
    # Attempt to create directories:
    if not os.path.exists(OUT_DIR + '/train/' + label):
        os.makedirs(OUT_DIR + '/train/' + label)
    if not os.path.exists(OUT_DIR + '/test/' + label):
        os.makedirs(OUT_DIR + '/test/' + label)
    if not os.path.exists(OUT_DIR + '/valid/' + label):
        os.makedirs(OUT_DIR + '/valid/' + label)
    if not os.path.exists(OUT_DIR_RAW + '/train/' + label):
        os.makedirs(OUT_DIR_RAW + '/train/' + label)
    if not os.path.exists(OUT_DIR_RAW + '/test/' + label):
        os.makedirs(OUT_DIR_RAW + '/test/' + label)
    if not os.path.exists(OUT_DIR_RAW + '/valid/' + label):
        os.makedirs(OUT_DIR_RAW + '/valid/' + label)
        
    train_path = os.path.join(OUT_DIR,'train/'+ label + '/')
    test_path  = os.path.join(OUT_DIR,'test/' + label + '/')
    valid_path = os.path.join(OUT_DIR,'valid/' + label + '/')
    train_path_raw = os.path.join(OUT_DIR_RAW,'train/'+ label + '/')
    test_path_raw  = os.path.join(OUT_DIR_RAW,'test/' + label + '/')
    valid_path_raw = os.path.join(OUT_DIR_RAW,'valid/' + label + '/')
    
    index = 0
    for img in im_list:
        if img in cropped_files:
            pic = Image.open(os.path.join(CROPPED_PATH,img))
            
            if (COPY_UNCROPPED):
                    pic2 = Image.open(os.path.join(UNCROPPED_PATH,img))
            
            if resize: #resize image
                out = pic.resize(size)
                if (COPY_UNCROPPED):
                    out2 = pic2.resize(size)
            else:
                out = pic
                if (COPY_UNCROPPED):
                    out2 = pic2
            
            if index < VAL_SIZE:
                out.save(os.path.join(valid_path,img))
            elif index < VAL_SIZE + TEST_SIZE:
                out.save(os.path.join(test_path,img))
            if index < VAL_SIZE and COPY_UNCROPPED:
                out2.save(os.path.join(valid_path_raw,img))
            elif index < VAL_SIZE + TEST_SIZE and COPY_UNCROPPED:
                out2.save(os.path.join(test_path_raw,img))    
            else:    #training set, rotate
                out.save(os.path.join(train_path,img.replace('.jpg','-0.jpg')))
                if COPY_UNCROPPED:
                    out2.save(os.path.join(train_path_raw,img.replace('.jpg','-0.jpg')))
                if preprocess:
                    for rot,mode,ext in zip([random.randint(0,359),random.randint(0,359),random.randint(0,359)],[random.choice(noise_modes),random.choice(noise_modes),random.choice(noise_modes)],[1,2,3]):
                        print(' ',img,rot,mode,ext,pic.mode)
                        if mode is not None:
                            Image.fromarray((add_noise(np.array(out.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic.mode).save(os.path.join(train_path,img.replace('.jpg','-'+str(ext)+'.jpg')))
                            if COPY_UNCROPPED:
                                Image.fromarray((add_noise(np.array(out2.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic2.mode).save(os.path.join(train_path_raw,img.replace('.jpg','-'+str(ext)+'.jpg')))
                        else:
                            out.rotate(rot).save(os.path.join(train_path,img.replace('.jpg','-'+str(ext)+'.jpg')))
                            if COPY_UNCROPPED:
                                out2.rotate(rot).save(os.path.join(train_path_raw,img.replace('.jpg','-'+str(ext)+'.jpg')))
                
        else:
            skipnum += 1
            index -= 1
            pass
        index += 1
    print(label, "skipped", skipnum)
    print(label, "total", index)
    
    total_skipped += skipnum
    
print("Images moved and rotated. Skipped ", total_skipped)

  3318-1.jpg 229 pepper 1 RGB
  3318-1.jpg 273 s&p 2 RGB
  3318-1.jpg 85 s&p 3 RGB
  8082-1.jpg 173 salt 1 RGB
  8082-1.jpg 70 pepper 2 RGB
  8082-1.jpg 248 None 3 RGB
  4687-1.jpg 31 None 1 RGB
  4687-1.jpg 189 salt 2 RGB
  4687-1.jpg 226 pepper 3 RGB
  6236-1.jpg 332 salt 1 RGB
  6236-1.jpg 9 salt 2 RGB
  6236-1.jpg 336 pepper 3 RGB
  5116-1.jpg 246 salt 1 RGB
  5116-1.jpg 318 salt 2 RGB
  5116-1.jpg 288 None 3 RGB
  6000-2.jpg 24 None 1 RGB
  6000-2.jpg 11 pepper 2 RGB
  6000-2.jpg 245 pepper 3 RGB
  5530-1.jpg 36 s&p 1 RGB
  5530-1.jpg 201 None 2 RGB
  5530-1.jpg 18 s&p 3 RGB
  6517-2.jpg 23 None 1 RGB
  6517-2.jpg 339 salt 2 RGB
  6517-2.jpg 334 pepper 3 RGB
  5667-3.jpg 233 salt 1 RGB
  5667-3.jpg 314 s&p 2 RGB
  5667-3.jpg 230 pepper 3 RGB
  7867-3.jpg 263 pepper 1 RGB
  7867-3.jpg 189 None 2 RGB
  7867-3.jpg 47 None 3 RGB
  8795-5.jpg 343 s&p 1 RGB
  8795-5.jpg 318 None 2 RGB
  8795-5.jpg 277 pepper 3 RGB
  5821-2.jpg 10 salt 1 RGB
  5821-2.jpg 211 s&p 2 RGB
  5821-2.jpg 157 s&

  6520-1.jpg 242 pepper 1 RGB
  6520-1.jpg 60 salt 2 RGB
  6520-1.jpg 71 salt 3 RGB
  2981-1.jpg 331 salt 1 RGB
  2981-1.jpg 341 pepper 2 RGB
  2981-1.jpg 164 s&p 3 RGB
  6360-1.jpg 185 s&p 1 RGB
  6360-1.jpg 279 salt 2 RGB
  6360-1.jpg 218 pepper 3 RGB
  532-1.jpg 94 None 1 RGB
  532-1.jpg 207 None 2 RGB
  532-1.jpg 57 None 3 RGB
  7287-4.jpg 359 pepper 1 RGB
  7287-4.jpg 190 salt 2 RGB
  7287-4.jpg 235 None 3 RGB
  4814-1.jpg 98 s&p 1 RGB
  4814-1.jpg 304 salt 2 RGB
  4814-1.jpg 289 salt 3 RGB
  5704-2.jpg 349 salt 1 RGB
  5704-2.jpg 45 s&p 2 RGB
  5704-2.jpg 296 pepper 3 RGB
  7856-1.jpg 49 pepper 1 RGB
  7856-1.jpg 121 None 2 RGB
  7856-1.jpg 193 pepper 3 RGB
  7087-3.jpg 356 s&p 1 RGB
  7087-3.jpg 301 s&p 2 RGB
  7087-3.jpg 31 s&p 3 RGB
  67-2.jpg 145 pepper 1 RGB
  67-2.jpg 222 pepper 2 RGB
  67-2.jpg 282 s&p 3 RGB
  2125-2.jpg 122 s&p 1 RGB
  2125-2.jpg 69 salt 2 RGB
  2125-2.jpg 177 None 3 RGB
  7864-2.jpg 331 salt 1 RGB
  7864-2.jpg 244 salt 2 RGB
  7864-2.jpg 16 s&p 3 RGB
  6

  6310-3.jpg 165 s&p 1 RGB
  6310-3.jpg 292 salt 2 RGB
  6310-3.jpg 305 None 3 RGB
  700-1.jpg 314 s&p 1 RGB
  700-1.jpg 25 s&p 2 RGB
  700-1.jpg 200 None 3 RGB
  5995-4.jpg 18 pepper 1 RGB
  5995-4.jpg 108 pepper 2 RGB
  5995-4.jpg 175 None 3 RGB
  4805-1.jpg 37 salt 1 RGB
  4805-1.jpg 359 pepper 2 RGB
  4805-1.jpg 254 pepper 3 RGB
  458-1.jpg 325 s&p 1 RGB
  458-1.jpg 177 None 2 RGB
  458-1.jpg 6 salt 3 RGB
  2774-1.jpg 272 salt 1 RGB
  2774-1.jpg 173 s&p 2 RGB
  2774-1.jpg 306 None 3 RGB
  5965-2.jpg 41 salt 1 RGB
  5965-2.jpg 339 None 2 RGB
  5965-2.jpg 222 pepper 3 RGB
  2296-1.jpg 130 salt 1 RGB
  2296-1.jpg 287 None 2 RGB
  2296-1.jpg 247 salt 3 RGB
  9610-1.jpg 240 None 1 RGB
  9610-1.jpg 336 salt 2 RGB
  9610-1.jpg 342 None 3 RGB
  6524-20.jpg 247 s&p 1 RGB
  6524-20.jpg 345 pepper 2 RGB
  6524-20.jpg 23 salt 3 RGB
  6005-3.jpg 230 salt 1 RGB
  6005-3.jpg 70 pepper 2 RGB
  6005-3.jpg 250 salt 3 RGB
  3419-1.jpg 26 None 1 RGB
  3419-1.jpg 172 None 2 RGB
  3419-1.jpg 91 s&p 3 RG

  6523-11.jpg 160 s&p 1 RGB
  6523-11.jpg 342 salt 2 RGB
  6523-11.jpg 277 pepper 3 RGB
  2848-1.jpg 39 None 1 RGB
  2848-1.jpg 325 None 2 RGB
  2848-1.jpg 135 s&p 3 RGB
  6579-1.jpg 181 None 1 RGB
  6579-1.jpg 42 salt 2 RGB
  6579-1.jpg 134 pepper 3 RGB
  6516-3.jpg 228 s&p 1 RGB
  6516-3.jpg 199 pepper 2 RGB
  6516-3.jpg 32 pepper 3 RGB
  8515-1.jpg 290 pepper 1 RGB
  8515-1.jpg 320 pepper 2 RGB
  8515-1.jpg 126 pepper 3 RGB
  6521-3.jpg 313 None 1 RGB
  6521-3.jpg 94 s&p 2 RGB
  6521-3.jpg 349 pepper 3 RGB
  8136-1.jpg 280 pepper 1 RGB
  8136-1.jpg 338 s&p 2 RGB
  8136-1.jpg 305 salt 3 RGB
  1599-1.jpg 347 None 1 RGB
  1599-1.jpg 40 salt 2 RGB
  1599-1.jpg 10 s&p 3 RGB
  5990-3.jpg 221 s&p 1 RGB
  5990-3.jpg 196 s&p 2 RGB
  5990-3.jpg 15 None 3 RGB
  6515-3.jpg 296 pepper 1 RGB
  6515-3.jpg 82 pepper 2 RGB
  6515-3.jpg 346 None 3 RGB
  5994-4.jpg 144 salt 1 RGB
  5994-4.jpg 44 s&p 2 RGB
  5994-4.jpg 42 s&p 3 RGB
  101-5.jpg 118 None 1 RGB
  101-5.jpg 342 s&p 2 RGB
  101-5.jpg 62 Non

  7409-1.jpg 83 s&p 1 RGB
  7409-1.jpg 116 s&p 2 RGB
  7409-1.jpg 290 None 3 RGB
  6331-3.jpg 225 pepper 1 RGB
  6331-3.jpg 312 pepper 2 RGB
  6331-3.jpg 310 None 3 RGB
  7903-1.jpg 166 None 1 RGB
  7903-1.jpg 228 salt 2 RGB
  7903-1.jpg 326 s&p 3 RGB
  5704-17.jpg 30 None 1 RGB
  5704-17.jpg 117 s&p 2 RGB
  5704-17.jpg 112 None 3 RGB
  2067-1.jpg 338 None 1 RGB
  2067-1.jpg 331 s&p 2 RGB
  2067-1.jpg 219 None 3 RGB
  2460-1.jpg 55 salt 1 RGB
  2460-1.jpg 274 salt 2 RGB
  2460-1.jpg 4 salt 3 RGB
  1605-1.jpg 223 salt 1 RGB
  1605-1.jpg 197 salt 2 RGB
  1605-1.jpg 50 salt 3 RGB
  6524-1.jpg 294 salt 1 RGB
  6524-1.jpg 317 pepper 2 RGB
  6524-1.jpg 269 salt 3 RGB
  1606-1.jpg 106 s&p 1 RGB
  1606-1.jpg 30 salt 2 RGB
  1606-1.jpg 86 pepper 3 RGB
  1037-3.jpg 20 pepper 1 RGB
  1037-3.jpg 52 s&p 2 RGB
  1037-3.jpg 19 pepper 3 RGB
  9565-2.jpg 71 salt 1 RGB
  9565-2.jpg 61 pepper 2 RGB
  9565-2.jpg 234 pepper 3 RGB
  6453-1.jpg 341 None 1 RGB
  6453-1.jpg 264 pepper 2 RGB
  6453-1.jpg 266 s&

...

  3351-1.jpg 17 s&p 3 RGB
  6237-3.jpg 356 salt 1 RGB
  6237-3.jpg 67 None 2 RGB
  6237-3.jpg 177 None 3 RGB
  5886-1.jpg 94 pepper 1 RGB
  5886-1.jpg 231 pepper 2 RGB
  5886-1.jpg 229 s&p 3 RGB
  739-1.jpg 145 s&p 1 RGB
  739-1.jpg 214 None 2 RGB
  739-1.jpg 355 s&p 3 RGB
  7165-5.jpg 224 pepper 1 RGB
  7165-5.jpg 215 None 2 RGB
  7165-5.jpg 139 pepper 3 RGB
  8363-1.jpg 278 pepper 1 RGB
  8363-1.jpg 56 s&p 2 RGB
  8363-1.jpg 233 salt 3 RGB
  3752-8.jpg 192 salt 1 RGB
  3752-8.jpg 34 salt 2 RGB
  3752-8.jpg 340 salt 3 RGB
  7783-4.jpg 85 pepper 1 RGB
  7783-4.jpg 285 s&p 2 RGB
  7783-4.jpg 309 None 3 RGB
  4273-3.jpg 181 None 1 RGB
  4273-3.jpg 100 None 2 RGB
  4273-3.jpg 257 None 3 RGB
  3251-3.jpg 337 None 1 RGB
  3251-3.jpg 13 None 2 RGB
  3251-3.jpg 265 s&p 3 RGB
  6209-3.jpg 80 pepper 1 RGB
  6209-3.jpg 74 salt 2 RGB
  6209-3.jpg 160 pepper 3 RGB
  6349-3.jpg 190 pepper 1 RGB
  6349-3.jpg 278 salt 2 RGB
  6349-3.jpg 94 pepper 3 RGB
  1277-1.jpg 243 None 1 RGB
  1277-1.jpg 338 sal

  3939-1.jpg 23 pepper 1 RGB
  3939-1.jpg 234 pepper 2 RGB
  3939-1.jpg 126 salt 3 RGB
  6837-3.jpg 354 None 1 RGB
  6837-3.jpg 83 s&p 2 RGB
  6837-3.jpg 181 pepper 3 RGB
  5730-1.jpg 94 None 1 RGB
  5730-1.jpg 312 pepper 2 RGB
  5730-1.jpg 274 None 3 RGB
  6787-1.jpg 309 salt 1 RGB
  6787-1.jpg 350 s&p 2 RGB
  6787-1.jpg 341 pepper 3 RGB
  5862-3.jpg 220 s&p 1 RGB
  5862-3.jpg 156 s&p 2 RGB
  5862-3.jpg 18 salt 3 RGB
  7359-1.jpg 185 None 1 RGB
  7359-1.jpg 89 salt 2 RGB
  7359-1.jpg 185 None 3 RGB
  6338-4.jpg 230 pepper 1 RGB
  6338-4.jpg 178 None 2 RGB
  6338-4.jpg 259 salt 3 RGB
  5137-5.jpg 244 salt 1 RGB
  5137-5.jpg 177 None 2 RGB
  5137-5.jpg 256 s&p 3 RGB
  6373-2.jpg 86 s&p 1 RGB
  6373-2.jpg 190 s&p 2 RGB
  6373-2.jpg 38 None 3 RGB
  5691-1.jpg 86 salt 1 RGB
  5691-1.jpg 235 None 2 RGB
  5691-1.jpg 99 None 3 RGB
  1242-2.jpg 346 pepper 1 RGB
  1242-2.jpg 202 s&p 2 RGB
  1242-2.jpg 246 None 3 RGB
  8223-4.jpg 177 None 1 RGB
  8223-4.jpg 160 salt 2 RGB
  8223-4.jpg 194 s&p 3 

  8583-5.jpg 79 None 1 RGB
  8583-5.jpg 150 salt 2 RGB
  8583-5.jpg 50 salt 3 RGB
  8548-2.jpg 130 None 1 RGB
  8548-2.jpg 19 salt 2 RGB
  8548-2.jpg 338 salt 3 RGB
  4192-1.jpg 145 salt 1 RGB
  4192-1.jpg 206 None 2 RGB
  4192-1.jpg 200 None 3 RGB
  4285-4.jpg 174 salt 1 RGB
  4285-4.jpg 255 salt 2 RGB
  4285-4.jpg 164 pepper 3 RGB
  1729-4.jpg 69 None 1 RGB
  1729-4.jpg 193 salt 2 RGB
  1729-4.jpg 214 pepper 3 RGB
  6148-1.jpg 161 None 1 RGB
  6148-1.jpg 29 s&p 2 RGB
  6148-1.jpg 245 salt 3 RGB
  8284-1.jpg 193 s&p 1 RGB
  8284-1.jpg 178 None 2 RGB
  8284-1.jpg 150 salt 3 RGB
  5615-1.jpg 244 pepper 1 RGB
  5615-1.jpg 8 None 2 RGB
  5615-1.jpg 144 s&p 3 RGB
  6789-10.jpg 234 pepper 1 RGB
  6789-10.jpg 90 s&p 2 RGB
  6789-10.jpg 186 s&p 3 RGB
  5808-4.jpg 147 pepper 1 RGB
  5808-4.jpg 129 pepper 2 RGB
  5808-4.jpg 315 salt 3 RGB
  6955-5.jpg 286 pepper 1 RGB
  6955-5.jpg 180 s&p 2 RGB
  6955-5.jpg 215 s&p 3 RGB
  9465-1.jpg 294 None 1 RGB
  9465-1.jpg 176 pepper 2 RGB
  9465-1.jpg 76 

  6147-7.jpg 314 salt 1 RGB
  6147-7.jpg 169 pepper 2 RGB
  6147-7.jpg 85 s&p 3 RGB
  8554-3.jpg 317 None 1 RGB
  8554-3.jpg 150 s&p 2 RGB
  8554-3.jpg 235 None 3 RGB
  5619-2.jpg 14 salt 1 RGB
  5619-2.jpg 123 salt 2 RGB
  5619-2.jpg 322 pepper 3 RGB
  7601-3.jpg 86 pepper 1 RGB
  7601-3.jpg 75 pepper 2 RGB
  7601-3.jpg 249 pepper 3 RGB
  9149-2.jpg 19 s&p 1 RGB
  9149-2.jpg 52 pepper 2 RGB
  9149-2.jpg 314 pepper 3 RGB
  1214-2.jpg 10 pepper 1 RGB
  1214-2.jpg 280 None 2 RGB
  1214-2.jpg 352 salt 3 RGB
  5692-1.jpg 344 None 1 RGB
  5692-1.jpg 150 salt 2 RGB
  5692-1.jpg 176 None 3 RGB
  6247-1.jpg 336 s&p 1 RGB
  6247-1.jpg 112 s&p 2 RGB
  6247-1.jpg 167 pepper 3 RGB
  9054-2.jpg 82 salt 1 RGB
  9054-2.jpg 152 pepper 2 RGB
  9054-2.jpg 105 salt 3 RGB
  4406-3.jpg 212 None 1 RGB
  4406-3.jpg 255 None 2 RGB
  4406-3.jpg 251 None 3 RGB
  5873-3.jpg 98 s&p 1 RGB
  5873-3.jpg 7 s&p 2 RGB
  5873-3.jpg 23 pepper 3 RGB
  8219-2.jpg 194 pepper 1 RGB
  8219-2.jpg 140 pepper 2 RGB
  8219-2.jpg 

  7787-1.jpg 73 pepper 1 RGB
  7787-1.jpg 202 pepper 2 RGB
  7787-1.jpg 206 s&p 3 RGB
  2169-2.jpg 223 pepper 1 RGB
  2169-2.jpg 31 pepper 2 RGB
  2169-2.jpg 264 pepper 3 RGB
  5687-1.jpg 81 pepper 1 RGB
  5687-1.jpg 86 salt 2 RGB
  5687-1.jpg 3 pepper 3 RGB
  1750-1.jpg 248 pepper 1 RGB
  1750-1.jpg 234 s&p 2 RGB
  1750-1.jpg 216 s&p 3 RGB
  8241-3.jpg 107 salt 1 RGB
  8241-3.jpg 34 pepper 2 RGB
  8241-3.jpg 225 s&p 3 RGB
  8614-1.jpg 296 pepper 1 RGB
  8614-1.jpg 224 salt 2 RGB
  8614-1.jpg 92 pepper 3 RGB
  6209-5.jpg 41 None 1 RGB
  6209-5.jpg 273 pepper 2 RGB
  6209-5.jpg 252 s&p 3 RGB
  5691-4.jpg 126 s&p 1 RGB
  5691-4.jpg 334 s&p 2 RGB
  5691-4.jpg 72 salt 3 RGB
  6712-3.jpg 23 s&p 1 RGB
  6712-3.jpg 298 None 2 RGB
  6712-3.jpg 156 None 3 RGB
  6948-2.jpg 135 None 1 RGB
  6948-2.jpg 298 salt 2 RGB
  6948-2.jpg 38 None 3 RGB
  2566-1.jpg 88 pepper 1 RGB
  2566-1.jpg 125 salt 2 RGB
  2566-1.jpg 142 s&p 3 RGB
  5925-6.jpg 221 salt 1 RGB
  5925-6.jpg 12 s&p 2 RGB
  5925-6.jpg 291 N

In [3]:
# Flip images over axes to even out training sets:
import fnmatch
import os
from PIL import Image
import random
import skimage

classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']
ABS_PATH_TRAIN = '/m2docs/res/data/train'
MULTIPLIER = 1.2 # how many times more images need to be created: mult * len(largest class)

c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN,c_name)), '*')) for c_name in classes]
print(classes)
print(c_count)

# add 25% more images to largest class,
# and bring the others to the same count
target = max(c_count) * 1.2
print("Target count: n =",target)

noise_modes = [None,'salt','pepper','s&p']
NOISE_AMOUNT = .01 # default amount
def add_noise(img, mode, noise_amount = .03):
    if mode is not None:
        gimg = skimage.util.random_noise(img, mode = mode, amount = random.uniform(0, noise_amount))
        return gimg
    else:
        print("oops, you shouldn't see this")

for c_name in classes:
    current = c_count[classes.index(c_name)]
    print(c_name, current, "->", target)
    im_list = os.listdir(os.path.join(ABS_PATH_TRAIN, c_name))
    while current < target:
        filename = random.choice(im_list)
        print(filename)
        out = Image.open(os.path.join(ABS_PATH_TRAIN, c_name, filename))
        
        rot = random.randint(0,359)
        mode = random.choice(noise_modes)
        print(filename,rot,mode,out.mode)
        print(filename.replace('.jpg','-x'+str(current)+'.jpg'))
        if mode is not None:
            Image.fromarray((add_noise(np.array(out.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic.mode).save(os.path.join(ABS_PATH_TRAIN, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
        else:
            out.rotate(rot).save(os.path.join(ABS_PATH_TRAIN, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
        current += 1

c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN,c_name)), '*')) for c_name in classes]
print(c_count)

if COPY_UNCROPPED:
    ABS_PATH_TRAIN_RAW = '/m2docs/res/data_raw/train'
    c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN_RAW,c_name)), '*')) for c_name in classes]
    print(classes)
    print(c_count)

    # add 25% more images to largest class,
    # and bring the others to the same count
    target = max(c_count) * 1.2
    print("Target count: n =",target)

    noise_modes = [None,'salt','pepper','s&p']
    NOISE_AMOUNT = .01 # default amount
    def add_noise(img, mode, noise_amount = .03):
        if mode is not None:
            gimg = skimage.util.random_noise(img, mode = mode, amount = random.uniform(0, noise_amount))
            return gimg
        else:
            print("oops, you shouldn't see this")

    for c_name in classes:
        current = c_count[classes.index(c_name)]
        print(c_name, current, "->", target)
        im_list = os.listdir(os.path.join(ABS_PATH_TRAIN_RAW, c_name))
        while current < target:
            filename = random.choice(im_list)
            print(filename)
            out = Image.open(os.path.join(ABS_PATH_TRAIN_RAW, c_name, filename))

            rot = random.randint(0,359)
            mode = random.choice(noise_modes)
            print(filename,rot,mode,out.mode)
            print(filename.replace('.jpg','-x'+str(current)+'.jpg'))
            if mode is not None:
                Image.fromarray((add_noise(np.array(out.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic.mode).save(os.path.join(ABS_PATH_TRAIN_RAW, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
            else:
                out.rotate(rot).save(os.path.join(ABS_PATH_TRAIN_RAW, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
            current += 1

    c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN_RAW,c_name)), '*')) for c_name in classes]
    print(c_count)




['Apis_mellifera', 'Bombus_impatiens', 'Bombus_auricomus', 'Bombus_bimaculatus', 'Bombus_griseocollis']
[3332, 6532, 1360, 2908, 5788]
Target count: n = 7838.4
Apis_mellifera 3332 -> 7838.4
6577-1-1.jpg
6577-1-1.jpg 352 pepper RGB
6577-1-1-x3332.jpg
1062-1-3.jpg
1062-1-3.jpg 307 s&p RGB
1062-1-3-x3333.jpg
2480-2-3.jpg
2480-2-3.jpg 303 pepper RGB
2480-2-3-x3334.jpg
3477-2-2.jpg
3477-2-2.jpg 326 s&p RGB
3477-2-2-x3335.jpg
2476-3-3.jpg
2476-3-3.jpg 232 salt RGB
2476-3-3-x3336.jpg
2382-2-2.jpg
2382-2-2.jpg 38 pepper RGB
2382-2-2-x3337.jpg
6516-4-2.jpg
6516-4-2.jpg 19 salt RGB
6516-4-2-x3338.jpg
8304-1-1.jpg
8304-1-1.jpg 274 salt RGB
8304-1-1-x3339.jpg
7975-2-2.jpg
7975-2-2.jpg 324 None RGB
7975-2-2-x3340.jpg
6579-1-3.jpg
6579-1-3.jpg 166 salt RGB
6579-1-3-x3341.jpg
6522-6-0.jpg
6522-6-0.jpg 73 None RGB
6522-6-0-x3342.jpg
8646-1-0.jpg
8646-1-0.jpg 263 None RGB
8646-1-0-x3343.jpg
5983-2-2.jpg
5983-2-2.jpg 54 pepper RGB
5983-2-2-x3344.jpg
1579-1-3.jpg
1579-1-3.jpg 49 pepper RGB
1579-1-3-x3345

4986-1-1.jpg
4986-1-1.jpg 224 None RGB
4986-1-1-x3473.jpg
975-1-3.jpg
975-1-3.jpg 72 pepper RGB
975-1-3-x3474.jpg
6521-3-2.jpg
6521-3-2.jpg 30 salt RGB
6521-3-2-x3475.jpg
6525-13-0.jpg
6525-13-0.jpg 157 salt RGB
6525-13-0-x3476.jpg
6392-1-2.jpg
6392-1-2.jpg 58 salt RGB
6392-1-2-x3477.jpg
5580-5-3.jpg
5580-5-3.jpg 340 pepper RGB
5580-5-3-x3478.jpg
5702-1-2.jpg
5702-1-2.jpg 264 pepper RGB
5702-1-2-x3479.jpg
8131-1-0.jpg
8131-1-0.jpg 73 salt RGB
8131-1-0-x3480.jpg
5839-2-3.jpg
5839-2-3.jpg 180 pepper RGB
5839-2-3-x3481.jpg
6334-3-1.jpg
6334-3-1.jpg 108 None RGB
6334-3-1-x3482.jpg
6604-1-1.jpg
6604-1-1.jpg 12 pepper RGB
6604-1-1-x3483.jpg
5136-18-0.jpg
5136-18-0.jpg 114 s&p RGB
5136-18-0-x3484.jpg
4459-1-3.jpg
4459-1-3.jpg 194 pepper RGB
4459-1-3-x3485.jpg
6950-1-3.jpg
6950-1-3.jpg 290 None RGB
6950-1-3-x3486.jpg
8136-1-1.jpg
8136-1-1.jpg 208 s&p RGB
8136-1-1-x3487.jpg
2907-1-0.jpg
2907-1-0.jpg 63 s&p RGB
2907-1-0-x3488.jpg
7903-1-3.jpg
7903-1-3.jpg 21 s&p RGB
7903-1-3-x3489.jpg
6524-5-0.j

871-1-1.jpg
871-1-1.jpg 335 pepper RGB
871-1-1-x3621.jpg
2644-1-2.jpg
2644-1-2.jpg 37 salt RGB
2644-1-2-x3622.jpg
5739-1-3.jpg
5739-1-3.jpg 0 None RGB
5739-1-3-x3623.jpg
6053-1-0.jpg
6053-1-0.jpg 269 None RGB
6053-1-0-x3624.jpg
2049-1-3.jpg
2049-1-3.jpg 236 pepper RGB
2049-1-3-x3625.jpg
5827-3-0.jpg
5827-3-0.jpg 181 pepper RGB
5827-3-0-x3626.jpg
1964-1-2.jpg
1964-1-2.jpg 217 pepper RGB
1964-1-2-x3627.jpg
3749-1-3.jpg
3749-1-3.jpg 83 salt RGB
3749-1-3-x3628.jpg
6310-6-2.jpg
6310-6-2.jpg 131 pepper RGB
6310-6-2-x3629.jpg
4609-1-3.jpg
4609-1-3.jpg 297 s&p RGB
4609-1-3-x3630.jpg
2862-1-1.jpg
2862-1-1.jpg 246 pepper RGB
2862-1-1-x3631.jpg
1062-1-1.jpg
1062-1-1.jpg 253 salt RGB
1062-1-1-x3632.jpg
6173-2-3.jpg
6173-2-3.jpg 308 s&p RGB
6173-2-3-x3633.jpg
6502-3-3.jpg
6502-3-3.jpg 256 None RGB
6502-3-3-x3634.jpg
7357-2-1.jpg
7357-2-1.jpg 82 None RGB
7357-2-1-x3635.jpg
1105-1-0.jpg
1105-1-0.jpg 5 s&p RGB
1105-1-0-x3636.jpg
2067-1-0.jpg
2067-1-0.jpg 286 salt RGB
2067-1-0-x3637.jpg
977-1-3.jpg
977

8153-1-0.jpg
8153-1-0.jpg 270 s&p RGB
8153-1-0-x3776.jpg
2171-3-2.jpg
2171-3-2.jpg 239 pepper RGB
2171-3-2-x3777.jpg
3126-1-3.jpg
3126-1-3.jpg 251 salt RGB
3126-1-3-x3778.jpg
6310-3-2.jpg
6310-3-2.jpg 75 salt RGB
6310-3-2-x3779.jpg
8090-1-2.jpg
8090-1-2.jpg 64 None RGB
8090-1-2-x3780.jpg
8082-1-2.jpg
8082-1-2.jpg 170 None RGB
8082-1-2-x3781.jpg
8305-2-3.jpg
8305-2-3.jpg 58 pepper RGB
8305-2-3-x3782.jpg
7190-1-0.jpg
7190-1-0.jpg 117 pepper RGB
7190-1-0-x3783.jpg
9517-1-0.jpg
9517-1-0.jpg 108 s&p RGB
9517-1-0-x3784.jpg
7866-4-1.jpg
7866-4-1.jpg 109 pepper RGB
7866-4-1-x3785.jpg
6525-9-3.jpg
6525-9-3.jpg 353 salt RGB
6525-9-3-x3786.jpg
7136-1-3.jpg
7136-1-3.jpg 73 pepper RGB
7136-1-3-x3787.jpg
2027-1-2.jpg
2027-1-2.jpg 247 s&p RGB
2027-1-2-x3788.jpg
5495-3-1.jpg
5495-3-1.jpg 350 salt RGB
5495-3-1-x3789.jpg
2480-2-3.jpg
2480-2-3.jpg 39 pepper RGB
2480-2-3-x3790.jpg
6524-1-2.jpg
6524-1-2.jpg 124 s&p RGB
6524-1-2-x3791.jpg
5968-1-0.jpg
5968-1-0.jpg 249 None RGB
5968-1-0-x3792.jpg
267-2-1.jpg

751-3-2.jpg
751-3-2.jpg 76 salt RGB
751-3-2-x3923.jpg
128-1-1.jpg
128-1-1.jpg 359 salt RGB
128-1-1-x3924.jpg
1757-1-2.jpg
1757-1-2.jpg 16 None RGB
1757-1-2-x3925.jpg
6499-3-3.jpg
6499-3-3.jpg 44 salt RGB
6499-3-3-x3926.jpg
6523-6-3.jpg
6523-6-3.jpg 357 salt RGB
6523-6-3-x3927.jpg
3134-1-3.jpg
3134-1-3.jpg 335 pepper RGB
3134-1-3-x3928.jpg
8063-1-0.jpg
8063-1-0.jpg 29 None RGB
8063-1-0-x3929.jpg
3134-1-3.jpg
3134-1-3.jpg 240 s&p RGB
3134-1-3-x3930.jpg
7906-3-3.jpg
7906-3-3.jpg 334 None RGB
7906-3-3-x3931.jpg
8170-1-1.jpg
8170-1-1.jpg 304 pepper RGB
8170-1-1-x3932.jpg
2126-4-1.jpg
2126-4-1.jpg 149 s&p RGB
2126-4-1-x3933.jpg
6310-3-3.jpg
6310-3-3.jpg 130 None RGB
6310-3-3-x3934.jpg
7151-1-3.jpg
7151-1-3.jpg 178 salt RGB
7151-1-3-x3935.jpg
7355-3-3.jpg
7355-3-3.jpg 76 pepper RGB
7355-3-3-x3936.jpg
5704-30-0.jpg
5704-30-0.jpg 299 s&p RGB
5704-30-0-x3937.jpg
1473-1-1.jpg
1473-1-1.jpg 177 salt RGB
1473-1-1-x3938.jpg
2480-2-0.jpg
2480-2-0.jpg 18 None RGB
2480-2-0-x3939.jpg
3142-1-3.jpg
3142-1-

...

8809-2-3.jpg
8809-2-3.jpg 124 s&p RGB
8809-2-3-x7143.jpg
6338-1-2.jpg
6338-1-2.jpg 335 None RGB
6338-1-2-x7144.jpg
6349-2-1.jpg
6349-2-1.jpg 178 None RGB
6349-2-1-x7145.jpg
9465-2-3.jpg
9465-2-3.jpg 258 pepper RGB
9465-2-3-x7146.jpg
8874-1-1.jpg
8874-1-1.jpg 76 pepper RGB
8874-1-1-x7147.jpg
2585-2-2.jpg
2585-2-2.jpg 28 s&p RGB
2585-2-2-x7148.jpg
8549-3-1.jpg
8549-3-1.jpg 297 pepper RGB
8549-3-1-x7149.jpg
5115-2-0.jpg
5115-2-0.jpg 202 None RGB
5115-2-0-x7150.jpg
8897-2-0.jpg
8897-2-0.jpg 14 s&p RGB
8897-2-0-x7151.jpg
7277-2-1.jpg
7277-2-1.jpg 331 s&p RGB
7277-2-1-x7152.jpg
8529-2-2.jpg
8529-2-2.jpg 145 salt RGB
8529-2-2-x7153.jpg
5146-2-2.jpg
5146-2-2.jpg 188 None RGB
5146-2-2-x7154.jpg
2268-1-1.jpg
2268-1-1.jpg 89 None RGB
2268-1-1-x7155.jpg
8607-2-1.jpg
8607-2-1.jpg 332 salt RGB
8607-2-1-x7156.jpg
9055-2-3.jpg
9055-2-3.jpg 245 s&p RGB
9055-2-3-x7157.jpg
2565-1-2.jpg
2565-1-2.jpg 200 pepper RGB
2565-1-2-x7158.jpg
7751-9-3.jpg
7751-9-3.jpg 15 s&p RGB
7751-9-3-x7159.jpg
7601-3-1.jpg
7601

6302-4-0.jpg
6302-4-0.jpg 14 pepper RGB
6302-4-0-x7286.jpg
431-1-0.jpg
431-1-0.jpg 199 pepper RGB
431-1-0-x7287.jpg
7376-3-1.jpg
7376-3-1.jpg 29 None RGB
7376-3-1-x7288.jpg
6874-1-1.jpg
6874-1-1.jpg 223 salt RGB
6874-1-1-x7289.jpg
5554-1-2.jpg
5554-1-2.jpg 38 s&p RGB
5554-1-2-x7290.jpg
8809-2-3.jpg
8809-2-3.jpg 339 s&p RGB
8809-2-3-x7291.jpg
7240-4-2.jpg
7240-4-2.jpg 94 salt RGB
7240-4-2-x7292.jpg
6712-1-3.jpg
6712-1-3.jpg 110 pepper RGB
6712-1-3-x7293.jpg
2627-1-2.jpg
2627-1-2.jpg 187 pepper RGB
2627-1-2-x7294.jpg
8864-1-3.jpg
8864-1-3.jpg 233 salt RGB
8864-1-3-x7295.jpg
8219-1-0.jpg
8219-1-0.jpg 328 pepper RGB
8219-1-0-x7296.jpg
5858-2-1.jpg
5858-2-1.jpg 142 pepper RGB
5858-2-1-x7297.jpg
4914-2-2.jpg
4914-2-2.jpg 24 salt RGB
4914-2-2-x7298.jpg
5188-2-1.jpg
5188-2-1.jpg 340 None RGB
5188-2-1-x7299.jpg
8379-2-1.jpg
8379-2-1.jpg 85 pepper RGB
8379-2-1-x7300.jpg
6302-4-0.jpg
6302-4-0.jpg 353 None RGB
6302-4-0-x7301.jpg
8715-1-2.jpg
8715-1-2.jpg 198 None RGB
8715-1-2-x7302.jpg
6747-4-3.jp

5830-4-0.jpg
5830-4-0.jpg 47 pepper RGB
5830-4-0-x7427.jpg
5749-2-1.jpg
5749-2-1.jpg 313 None RGB
5749-2-1-x7428.jpg
8838-1-0.jpg
8838-1-0.jpg 339 s&p RGB
8838-1-0-x7429.jpg
4082-1-2.jpg
4082-1-2.jpg 74 salt RGB
4082-1-2-x7430.jpg
5123-1-3.jpg
5123-1-3.jpg 259 s&p RGB
5123-1-3-x7431.jpg
4980-1-1.jpg
4980-1-1.jpg 22 pepper RGB
4980-1-1-x7432.jpg
9157-1-3.jpg
9157-1-3.jpg 103 salt RGB
9157-1-3-x7433.jpg
6363-1-1.jpg
6363-1-1.jpg 138 pepper RGB
6363-1-1-x7434.jpg
7124-3-3.jpg
7124-3-3.jpg 175 pepper RGB
7124-3-3-x7435.jpg
1303-1-1.jpg
1303-1-1.jpg 352 salt RGB
1303-1-1-x7436.jpg
7626-4-1.jpg
7626-4-1.jpg 355 pepper RGB
7626-4-1-x7437.jpg
9353-3-2.jpg
9353-3-2.jpg 257 s&p RGB
9353-3-2-x7438.jpg
4170-1-0.jpg
4170-1-0.jpg 247 salt RGB
4170-1-0-x7439.jpg
4797-2-1.jpg
4797-2-1.jpg 105 pepper RGB
4797-2-1-x7440.jpg
7059-1-3.jpg
7059-1-3.jpg 195 s&p RGB
7059-1-3-x7441.jpg
7451-3-3.jpg
7451-3-3.jpg 168 s&p RGB
7451-3-3-x7442.jpg
4958-1-2.jpg
4958-1-2.jpg 115 pepper RGB
4958-1-2-x7443.jpg
424-2-0.

6209-5-3.jpg 87 pepper RGB
6209-5-3-x7571.jpg
5694-1-3.jpg
5694-1-3.jpg 19 None RGB
5694-1-3-x7572.jpg
3059-3-2.jpg
3059-3-2.jpg 198 s&p RGB
3059-3-2-x7573.jpg
5697-6-0.jpg
5697-6-0.jpg 351 None RGB
5697-6-0-x7574.jpg
7236-1-3.jpg
7236-1-3.jpg 240 salt RGB
7236-1-3-x7575.jpg
9149-2-2.jpg
9149-2-2.jpg 337 None RGB
9149-2-2-x7576.jpg
8670-2-0.jpg
8670-2-0.jpg 358 pepper RGB
8670-2-0-x7577.jpg
2969-2-2.jpg
2969-2-2.jpg 137 pepper RGB
2969-2-2-x7578.jpg
4476-6-2.jpg
4476-6-2.jpg 145 salt RGB
4476-6-2-x7579.jpg
275-2-1.jpg
275-2-1.jpg 251 None RGB
275-2-1-x7580.jpg
5554-6-2.jpg
5554-6-2.jpg 349 s&p RGB
5554-6-2-x7581.jpg
1746-1-0.jpg
1746-1-0.jpg 301 salt RGB
1746-1-0-x7582.jpg
1227-1-3.jpg
1227-1-3.jpg 248 None RGB
1227-1-3-x7583.jpg
8760-1-2.jpg
8760-1-2.jpg 120 pepper RGB
8760-1-2-x7584.jpg
6286-1-0.jpg
6286-1-0.jpg 33 pepper RGB
6286-1-0-x7585.jpg
7210-3-2.jpg
7210-3-2.jpg 211 s&p RGB
7210-3-2-x7586.jpg
739-1-1.jpg
739-1-1.jpg 139 salt RGB
739-1-1-x7587.jpg
5767-2-0.jpg
5767-2-0.jpg 344

6731-2-3.jpg 102 salt RGB
6731-2-3-x7718.jpg
1746-1-1.jpg
1746-1-1.jpg 249 pepper RGB
1746-1-1-x7719.jpg
8219-1-0.jpg
8219-1-0.jpg 351 None RGB
8219-1-0-x7720.jpg
6144-4-3.jpg
6144-4-3.jpg 88 pepper RGB
6144-4-3-x7721.jpg
6747-4-2.jpg
6747-4-2.jpg 168 None RGB
6747-4-2-x7722.jpg
8332-1-3.jpg
8332-1-3.jpg 241 s&p RGB
8332-1-3-x7723.jpg
6345-5-3.jpg
6345-5-3.jpg 255 s&p RGB
6345-5-3-x7724.jpg
6107-1-2.jpg
6107-1-2.jpg 289 s&p RGB
6107-1-2-x7725.jpg
7944-4-1.jpg
7944-4-1.jpg 263 salt RGB
7944-4-1-x7726.jpg
4042-1-2.jpg
4042-1-2.jpg 146 None RGB
4042-1-2-x7727.jpg
7783-2-1.jpg
7783-2-1.jpg 273 salt RGB
7783-2-1-x7728.jpg
6168-1-1.jpg
6168-1-1.jpg 79 None RGB
6168-1-1-x7729.jpg
6650-2-1.jpg
6650-2-1.jpg 229 pepper RGB
6650-2-1-x7730.jpg
6882-1-3.jpg
6882-1-3.jpg 256 s&p RGB
6882-1-3-x7731.jpg
9613-1-2.jpg
9613-1-2.jpg 4 None RGB
9613-1-2-x7732.jpg
3740-1-3.jpg
3740-1-3.jpg 239 pepper RGB
3740-1-3-x7733.jpg
7430-3-2.jpg
7430-3-2.jpg 226 pepper RGB
7430-3-2-x7734.jpg
5551-2-2.jpg
5551-2-2.jpg

## Train classifier
Based on [this](https://nextjournal.com/gkoehler/pytorch-mnist) pytorch tutorial

Load images and train classifier.

In [4]:
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import os, datetime, math

from torch.utils.tensorboard import SummaryWriter 
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp 
length = 256
skip_all = False

classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']
#classes = ['Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

ABS_PATH_TRAIN = '/m2docs/res/data/train'
ABS_PATH_VALID = '/m2docs/res/data/valid'
ABS_PATH_TEST = '/m2docs/res/data/test'
ABS_PATH_TRAIN_RAW = '/m2docs/res/data_raw/train'
ABS_PATH_VALID_RAW = '/m2docs/res/data_raw/valid'
ABS_PATH_TEST_RAW = '/m2docs/res/data_raw/test'

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def load(dir_name, batch_size, shuffle = False):
    return(
        #create a data loader
        torch.utils.data.DataLoader(
            datasets.ImageFolder(root = dir_name, transform = transforms.ToTensor()),
            batch_size = batch_size,
            num_workers = 2,
            shuffle = shuffle
        )
    )

class Net(nn.Module):
    def __init__(self, hparams):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(p = hparams[HP_DROPOUT])
        
        self.conv1 = nn.Conv2d(3, int(hparams[HP_NUM_UNITS]/8), 5)
        self.conv2 = nn.Conv2d(int(hparams[HP_NUM_UNITS]/8), int(hparams[HP_NUM_UNITS]/4), 5)
        self.conv3 = nn.Conv2d(int(hparams[HP_NUM_UNITS]/4), int(hparams[HP_NUM_UNITS]/2), 5)
        self.conv4 = nn.Conv2d(int(hparams[HP_NUM_UNITS]/2), hparams[HP_NUM_UNITS], 5)
        
        self.fc1 = nn.Linear(hparams[HP_NUM_UNITS]*28*28, length)
        self.fc2 = nn.Linear(length, int(length/2))
        #self.fc3 = nn.Linear(int(length/2), int(length/4))
        self.fc3 = nn.Linear(int(length/2), 5)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = self.dropout(F.relu(self.pool(self.conv2(x))))
        x = self.dropout(F.relu(self.pool(self.conv3(x))))
        x = (F.relu(self.pool(self.conv4(x))))
        #print(x.size())
        x = x.view(x.size(0),-1)
        #print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(self.fc3(x))
        return x

def validate(network,device,load_valid,optimizer,criterion = nn.CrossEntropyLoss()):
    network.train().to(device)
    correct = 0
    valid_loss = 0
    for index, data in enumerate(load_valid, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        valid_loss += loss.item()

        # gather accuracy stats:
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).float().sum().item()
    print(" Validation correct: {} / {}".format(correct,len(load_valid.dataset)))  
    accuracy = 100 * correct / len(load_valid.dataset)
    valid_loss = valid_loss / len(load_valid.dataset)
    return accuracy, valid_loss

def test(network,device,load_test,criterion = nn.CrossEntropyLoss()):
    network.eval().to(device)
    correct = 0
    valid_loss = 0
    with torch.no_grad():
        for index, data in enumerate(load_test, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            # forward + backward + optimize
            outputs = network(inputs)
            valid_loss += criterion(outputs, labels).item()

            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
    print(" Testing correct: {} / {}".format(correct,len(load_test.dataset)))        
    accuracy = 100 * correct / len(load_test.dataset)
    valid_loss = valid_loss / len(load_test.dataset)
    return accuracy , valid_loss

def train(max_epochs, min_epochs, epoch_stretch, batch_size, train_path, valid_path, test_path, labels, hparams, writer):
    epochs = max_epochs
    class_names = labels
    num_classes = len(class_names)
    train_batch = batch_size
    test_batch = 128
    SAVE_PATH = '/m2docs/res/trained_models/model'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device: {}".format(device))
    net = Net(hparams).to(device)
    print(net)
    
    tag = datetime.datetime.now().strftime(".%Y%m%d-%H%M%S")

    criterion = nn.CrossEntropyLoss()
    
    # Choose optimizer: (from ['adam','sgd','adagrad'])
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    if (hparams[HP_OPTIMIZER] == 'adam'):
        optimizer = optim.Adam(net.parameters(), lr = .001)
    elif (hparams[HP_OPTIMIZER] == 'sgd'):
        optimizer = optim.SGD(net.parameters(), lr = .001, momentum=0.9)
    elif (hparams[HP_OPTIMIZER] == 'adagrad'):
        optimizer = optim.Adagrad(net.parameters(), lr = .001,)
    else:
        # default
        optimizer = optim.Adam(net.parameters(), lr = 0.001)

    torch.manual_seed(417)
    
    load_train = load(train_path, batch_size, shuffle=True)
    load_valid = load(valid_path, batch_size, shuffle=True)
    load_test  = load(test_path, batch_size, shuffle=True)

    validation_accuracies = []
    best_epoch = 0
    epoch = 0
    killed = False
    while (epoch <= best_epoch + epoch_stretch or epoch < min_epochs) and epoch < max_epochs and not killed:
        net.train()
        run_loss = 0.0
        sum_loss = 0.0
        count = 0
        correct = 0.0
        categorical_correct = [0.0 for i in range(num_classes)]
        for index, data in enumerate(load_train, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
            for i in range(num_classes):
                categorical_correct[i] += ((predicted==i) == (labels==i)).float().sum().item()
                #print(classes[i],categorical_correct[i]/((index+1)*batch_size))
            
            # print statistics
            run_loss += loss.item()
            sum_loss += loss.item()
            count += 1 
            if index % 200 == 0:    # print every 200 mini-batches
                print('  Epoch: {} [{}/{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.6f}%'.format(
                    epoch, int(correct), (index + 1) * batch_size, len(load_train.dataset),
                    100. * index / len(load_train), loss.item(), 100. * correct / ((index+1) * batch_size)))
                run_loss = 0.0
        accuracy = 100. * correct / len(load_train.dataset)
        print('Epoch: {}\tLoss: {:.6f}\tAcc: {:.6f}'.format(
                epoch, sum_loss/count, accuracy))
        writer.add_scalar("Loss/train", sum_loss/count, epoch)
        writer.add_scalar("Acc/train", accuracy, epoch)
        
        modules_list = iter(net.named_modules())
        next(modules_list)
        for module in modules_list:
            try:
                writer.add_histogram("Model/"+module[0]+".weights", module[1].weight, epoch)
                writer.add_histogram("Model/"+module[0]+".bias", module[1].bias, epoch)
            except:
                pass
        # Categorical accuracy:
        for i in range(num_classes):
            writer.add_scalar("Acc/" + classes[i],categorical_correct[i]/len(load_train.dataset), epoch)
        torch.save(net.state_dict(), SAVE_PATH + tag + "-progress")
        
        # get validation accuracy: 
        valid_acc, valid_loss = validate(net, device, load_valid, optimizer, criterion)
        writer.add_scalar("Loss/valid", valid_loss, epoch)
        writer.add_scalar("Acc/valid", valid_acc, epoch)
        print('Validation: acc: {:.6f}%\tloss: {:.6f}'.format(
                valid_acc, valid_loss))
        validation_accuracies.append(valid_acc)
        best_epoch = validation_accuracies.index(max(validation_accuracies))
        
        # get test accuracy: 
        test_acc, test_loss = test(net, device, load_test, criterion)
        writer.add_scalar("Loss/test", test_loss, epoch)
        writer.add_scalar("Acc/test", test_acc, epoch)
        print('Testing: acc: {:.6f}%\tloss: {:.6f}'.format(
                test_acc, test_loss))
        
        # this is the best epoch so far, save these weights:
        if (best_epoch == epoch):
            torch.save(net.state_dict(), SAVE_PATH + tag + "-best")
        if (epoch >= 2):
            if (validation_accuracies[epoch] == 100/num_classes):
                killed = True
                print("[!] This run has failed, accuracies are bad. Aborting.")
        
        print('Best: {} @ {:.6f}% -> epoch target {}'.format(best_epoch,validation_accuracies[best_epoch],max([best_epoch+epoch_stretch,min_epochs])))
        epoch += 1
        
    print('Done training.')
    torch.save(net.state_dict(), SAVE_PATH + tag + "-final")
    
    ## TODO: test here

    prediction_list = torch.zeros(0,dtype=torch.long).to(device)
    label_list = torch.zeros(0,dtype=torch.long).to(device)
    
    ## Testing
    net.eval().to(device)
    correct = 0
    valid_loss = 0
    with torch.no_grad():
        for index, data in enumerate(load_test, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = net(inputs)
            valid_loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs.data, 1)
            prediction_list = torch.cat([prediction_list, predicted.view(-1)])  
            label_list = torch.cat([label_list, labels.view(-1)])
            #print("Predictions, ground:")
            #print(prediction_list)
            #print(label_list)
            correct += (predicted == labels).float().sum().item()
    print(" Testing correct: {} / {}".format(correct,len(load_test.dataset)))        
    t_acc = 100 * correct / len(load_test.dataset)
    t_loss = valid_loss / len(load_test.dataset)

    
    matrix = confusion_matrix(label_list.cpu().numpy(), prediction_list.cpu().numpy())
    print(matrix)
    fig, ax = plt.subplots()
    im = ax.imshow(matrix)
    # We want to show all ticks...
    ax.set_xticks(np.arange(len(classes)))
    ax.set_yticks(np.arange(len(classes)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(classes)):
        for j in range(len(classes)):
            text = ax.text(j, i, "{:4.2f}".format(matrix[i, j]/sum(matrix[i])),
                           ha="center", va="center", color="w")

    ax.set_title("Testing confusion matrix (n = {})".format(len(load_test.dataset)))

    plt.show()
    writer.add_figure('Testing/conf',fig)
    
    class_accuracy=100*matrix.diagonal() / matrix.sum(1)
    #print(classes)
    #print(class_accuracy)
    for i in range(len(classes)):
        print("{}: {:.4f}".format(classes[i],class_accuracy[i]))
    print("Best val_acc: {:6.4f}".format(max(validation_accuracies)))
    return max(validation_accuracies)

# magic here.
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        run_writer = SummaryWriter(log_dir = run_dir)            
        if (hparams[HP_YOLOCROPPED]):
            train_path = ABS_PATH_TRAIN
            valid_path = ABS_PATH_VALID
            test_path = ABS_PATH_TEST
        else:
            train_path = ABS_PATH_TRAIN_RAW
            valid_path = ABS_PATH_VALID_RAW
            test_path = ABS_PATH_TEST_RAW
        
        accuracy = train(max_epochs = 100, min_epochs = 10, epoch_stretch = 5, batch_size = 32, train_path, valid_path, test_path, labels = classes, hparams = hparams, writer = run_writer)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

if (skip_all is False): 
    HP_NUM_UNITS = hp.HParam('channels', hp.Discrete([128, 256]))
    HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
    #HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['sgd','adagrad']))
    HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['sgd']))
    HP_YOLOCROPPED = hp.HParam('processed (yolo)', hp.Discrete([True, False]))
    METRIC_ACCURACY = 'accuracy'
    
    hpdirname = 'runs/' + (datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ':hparam_tuning')
    with tf.summary.create_file_writer(hpdirname).as_default():
        hp.hparams_config(
            hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
            metrics=[hp.Metric(METRIC_ACCURACY, display_name='best accuracy (validation)')]
        )
    
    session_num = 0
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            for optimizer in HP_OPTIMIZER.domain.values:
                for yolo in HP_YOLOCROPPED.domain.values:
                    hparams = {
                        HP_NUM_UNITS: num_units,
                        HP_DROPOUT: float("%0.2f"%float(dropout_rate)),
                        HP_OPTIMIZER: optimizer,
                        HP_YOLOCROPPED: yolo,
                    }
                    torch.cuda.empty_cache()
                    run_name = "run-%d" % session_num
                    print("-> Starting trial %s" % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    if hparams[HP_YOLOCROPPED]:
                        run(os.path.join(hpdirname, '_' + str(session_num) + '_true'), hparams)
                    else:
                        run(os.path.join(hpdirname, '_' + str(session_num)) + '_false', hparams)
                    session_num += 1

SyntaxError: positional argument follows keyword argument (<ipython-input-4-8c1d0a2543c4>, line 321)

# 2 stage classifier

1. Train to detect *Apis Mellifera* vs. *Bombus Auricomus* vs. a merged set of the other classes.
2. Train a second model to tell apart the other three classes.'
3. Note that both of these datasets will by default be unbalanced unless something is changed above.
4. When running detections, if the first model's confidence is below a threshold, apply the second model to see if there is an improved prediction.

`todo: balance datasets`

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import os, datetime, math

from torch.utils.tensorboard import SummaryWriter 
length = 512
var_droupout = 0.2
skip_all = True
tag1 = ':' + str(length) + '_STAGE-1_' + str(var_droupout)
tag2 = ':' + str(length) + '_STAGE-2_' + str(var_droupout)
writer1 = SummaryWriter(log_dir = os.path.join('runs/',(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + tag1)))
writer2 = SummaryWriter(log_dir = os.path.join('runs/',(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + tag2)))
print("tensorboard writing"+tag1)
print("tensorboard writing"+tag2)

classes1 = ['Apis_mellifera','Bombus_impatiens','Merged']
classes2 = ['Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

ABS_PATH_TRAIN1 = '/m2docs/res/data1/train'
ABS_PATH_VALID1 = '/m2docs/res/data1/valid'
ABS_PATH_TEST1 = '/m2docs/res/data1/test'
ABS_PATH_TRAIN2 = '/m2docs/res/data2/train'
ABS_PATH_VALID2 = '/m2docs/res/data2/valid'
ABS_PATH_TEST2 = '/m2docs/res/data2/test'

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def load(dir_name, batch_size, shuffle = False):
    return(
        #create a data loader
        torch.utils.data.DataLoader(
            datasets.ImageFolder(root = dir_name, transform = transforms.ToTensor()),
            batch_size = batch_size,
            num_workers = 2,
            shuffle = shuffle
        )
    )

# Layer/network 1:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(p = var_droupout)
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.conv4 = nn.Conv2d(64, 128, 5)
        
        self.fc1 = nn.Linear(128*28*28, length)
        self.fc2 = nn.Linear(length, int(length/2))
        #self.fc3 = nn.Linear(int(length/2), int(length/4))
        self.fc3 = nn.Linear(int(length/2), 5)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = self.dropout(F.relu(self.pool(self.conv2(x))))
        x = self.dropout(F.relu(self.pool(self.conv3(x))))
        x = (F.relu(self.pool(self.conv4(x))))
        #print(x.size())
        x = x.view(x.size(0),-1)
        #print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(self.fc3(x))
        return x
    
# Layer/network 2:    
class Net2(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d(p = var_droupout)
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.conv4 = nn.Conv2d(64, 128, 5)
        
        self.fc1 = nn.Linear(128*28*28, length)
        self.fc2 = nn.Linear(length, int(length/2))
        #self.fc3 = nn.Linear(int(length/2), int(length/4))
        self.fc3 = nn.Linear(int(length/2), 5)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = self.dropout(F.relu(self.pool(self.conv2(x))))
        x = self.dropout(F.relu(self.pool(self.conv3(x))))
        x = (F.relu(self.pool(self.conv4(x))))
        #print(x.size())
        x = x.view(x.size(0),-1)
        #print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(self.fc3(x))
        return x

def validate(network,device,load_valid,criterion = nn.CrossEntropyLoss()):
    network.eval().to(device)
    correct = 0
    valid_loss = 0
    with torch.no_grad():
        for index, data in enumerate(load_valid, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            # forward + backward + optimize
            outputs = network(inputs)
            valid_loss += criterion(outputs, labels).item()

            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
    accuracy = 100 * correct / len(load_valid.dataset)
    valid_loss = valid_loss / len(load_valid.dataset)
    return accuracy , valid_loss

def train(max_epochs = 50, min_epochs = 5, epoch_stretch = 5, train_path = ABS_PATH_TRAIN, valid_path = ABS_PATH_VALID, test_path = ABS_PATH_TEST, labels = classes, batch_size = 24):
    epochs = max_epochs
    class_names = labels
    num_classes = len(class_names)
    train_batch = batch_size
    test_batch = 128
    SAVE_PATH = '/m2docs/res/models'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device: {}".format(device))
    net = Net().to(device)
    print(net)

    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr = 0.001)

    torch.manual_seed(417)
    
    load_train = load(train_path, batch_size, shuffle=True)
    load_valid = load(valid_path, batch_size, shuffle=True)
    load_test  = load(test_path, batch_size, shuffle=True)

    validation_accuracies = []
    best_epoch = 0
    epoch = 0
    while (epoch <= best_epoch + epoch_stretch or epoch < min_epochs) and epoch < max_epochs:
        net.train()
        run_loss = 0.0
        sum_loss = 0.0
        count = 0
        correct = 0.0
        categorical_correct = [0.0 for i in range(num_classes)]
        for index, data in enumerate(load_train, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
            for i in range(num_classes):
                categorical_correct[i] += ((predicted==i) == (labels==i)).float().sum().item()
                #print(classes[i],categorical_correct[i]/((index+1)*batch_size))
            
            # print statistics
            run_loss += loss.item()
            sum_loss += loss.item()
            count += 1 
            if index % 200 == 0:    # print every 200 mini-batches
                print('  Epoch: {} [{}/{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.6f}%'.format(
                    epoch, int(correct), (index + 1) * batch_size, len(load_train.dataset),
                    100. * index / len(load_train), loss.item(), 100. * correct / ((index+1) * batch_size)))
                run_loss = 0.0
        accuracy = 100. * correct / len(load_train.dataset)
        print('Epoch: {}\tLoss: {:.6f}\tAcc: {:.6f}'.format(
                epoch, sum_loss/count, accuracy))
        writer.add_scalar("Loss/train", sum_loss/count, epoch)
        writer.add_scalar("Acc/train", accuracy, epoch)
        
        modules_list = iter(net.named_modules())
        next(modules_list)
        for module in modules_list:
            try:
                writer.add_histogram("Model/"+module[0]+".weights", module[1].weight, epoch)
                writer.add_histogram("Model/"+module[0]+".bias", module[1].bias, epoch)
            except:
                pass
        # Categorical accuracy:
        for i in range(num_classes):
            writer.add_scalar("Acc/" + classes[i],categorical_correct[i]/len(load_train.dataset), epoch)
        torch.save(net.state_dict(), SAVE_PATH+"_progress")
        
        # get validation accuracy: 
        valid_acc, valid_loss = validate(net, device, load_valid, criterion)
        writer.add_scalar("Loss/valid", valid_loss, epoch)
        writer.add_scalar("Acc/valid", valid_acc, epoch)
        print('Validation: acc: {:.6f}%\tloss: {:.6f}'.format(
                valid_acc, valid_loss))
        validation_accuracies.append(valid_acc)
        best_epoch = validation_accuracies.index(max(validation_accuracies))
        print('Best: {} @ {:.6f}% -> epoch target {}'.format(best_epoch,validation_accuracies[best_epoch],max([best_epoch+epoch_stretch,min_epochs])))
        epoch += 1
        
    print('Done training.')
    torch.save(net.state_dict(), SAVE_PATH)

    prediction_list = torch.zeros(0,dtype=torch.long).to(device)
    label_list = torch.zeros(0,dtype=torch.long).to(device)
              
    with torch.no_grad():
        for data in load_test:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
              
            prediction_list = torch.cat([prediction_list, predicted.view(-1)])  
            label_list = torch.cat([label_list, labels.view(-1)])
    
    matrix = confusion_matrix(label_list.cpu().numpy(), prediction_list.cpu().numpy())
    print(matrix)
    fig, ax = plt.subplots()
    im = ax.imshow(matrix)
    # We want to show all ticks...
    ax.set_xticks(np.arange(len(classes)))
    ax.set_yticks(np.arange(len(classes)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(classes)):
        for j in range(len(classes)):
            text = ax.text(j, i, matrix[i, j],
                           ha="center", va="center", color="w")

    ax.set_title("Testing confusion matrix")

    plt.show()
    writer.add_figure('Testing/conf',fig)
    
    class_accuracy=100*matrix.diagonal() / matrix.sum(1)
    print(classes)
    print(class_accuracy)

if (skip_all is False):
    # setup: create 2-stage directories:
    # data1: first stage: mellifera, impatiens, 3-merged
    %mkdir data1
    %rm -r data1/*
    %mkdir data1/train
    %cp -r data/train/Apis_mellifera data1/train/Apis_mellifera
    %cp -r data/train/Bombus_auricomus data1/train/Bombus_auricomus
    %cp -r data/train/Bombus_bimaculatus data1/train/Merged
    %cp data/train/Bombus_griseocollis/* data1/train/Merged
    %cp data/train/Bombus_impatiens/* data1/train/Merged
    %mkdir data1/valid
    %cp -r data/valid/Apis_mellifera data1/valid/Apis_mellifera
    %cp -r data/valid/Bombus_auricomus data1/valid/Bombus_auricomus
    %cp -r data/valid/Bombus_bimaculatus data1/valid/Merged
    %cp data/valid/Bombus_griseocollis/* data1/valid/Merged
    %cp data/valid/Bombus_impatiens/* data1/valid/Merged
    %mkdir data1/test
    %cp -r data/test/Apis_mellifera data1/test/Apis_mellifera
    %cp -r data/test/Bombus_auricomus data1/test/Bombus_auricomus
    %cp -r data/test/Bombus_bimaculatus data1/test/Merged
    %cp data/test/Bombus_griseocollis/* data1/test/Merged
    %cp data/test/Bombus_impatiens/* data1/test/Merged

    # data2: auricomus vs bimaculatus vs griseocollis
    %mkdir data2
    %rm -r data2/*
    %mkdir data2/train
    %cp -r data/train/Bombus_bimaculatus data2/train/Bombus_bimaculatus
    %cp -r data/train/Bombus_griseocollis data2/train/Bombus_griseocollis
    %cp -r data/train/Bombus_impatiens data2/train/Bombus_impatiens
    %mkdir data2/valid
    %cp -r data/valid/Bombus_bimaculatus data2/valid/Bombus_bimaculatus
    %cp -r data/valid/Bombus_griseocollis data2/valid/Bombus_griseocollis
    %cp -r data/valid/Bombus_impatiens data2/valid/Bombus_impatiens
    %mkdir data2/test
    %cp -r data/test/Bombus_bimaculatus data2/test/Bombus_bimaculatus
    %cp -r data/test/Bombus_griseocollis data2/test/Bombus_griseocollis
    %cp -r data/test/Bombus_impatiens data2/test/Bombus_impatiens
    
    # train
    train(max_epochs = 160, min_epochs = 70, epoch_stretch = 15, train_path = ABS_PATH_TRAIN, valid_path = ABS_PATH_VALID, test_path = ABS_PATH_TEST, labels = classes, batch_size = 32)